In [1]:
# here we link our notebook to our gdrive space
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 7.3 MB/s 


In [3]:
!pip install pycountry-convert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 45.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 298 kB 39.3 MB/s 
     |████████████████████████████████| 209 kB 67.7 MB/s 
     |████████████████████████████████| 297 kB 67.0 MB/s 
     |████████████████████████████████| 297 kB 65.2 MB/s 
     |████████████████████████████████| 297 kB 78.5 MB/s 
     |████████████████████████████████| 296 kB 63.7 MB/s 
     |████████████████████████████████| 296 kB 62.7 MB/s 
     |████████████████████████████████| 280 kB 63.7 MB/s 
     |████████████████████████████████| 280 kB 48.7 MB/s 
     |████████████████████████████████| 280 kB 65.4 MB/s 
     |████████████████████████████████| 280 kB 59.3 MB/s 
     |████████████████████████████████| 279 kB 71.7 MB/s 
     |██████████████

In [4]:
from sklearn.model_selection import GridSearchCV
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import LeaveOneOutEncoder
from keras.models import Sequential
from keras.layers import Dense
import os
import pycountry_convert as pc
import sklearn
import pandas as pd
import numpy as np
import json
import random, string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import category_encoders as ce
%matplotlib inline

In [5]:
# load files
train = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/Test.csv')
variableDefinitions = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/VariableDefinitions.csv')
sampleSubmission = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/SampleSubmission.csv')

In [6]:
# data preprocessing
# Feature Engineering

# to map country to continent
def country_to_continent(df):
  df = df.replace('SWIZERLAND', 'SWITZERLAND')
  df = df.replace('MORROCO', 'MOROCCO')
  df = df.replace('UAE', 'UNITED ARAB EMIRATES')
  df = df.replace('KOREA', 'SOUTH KOREA')
  df = df.replace('DRC', 'DEMOCRATIC REPUBLIC OF THE CONGO')
  df = df.replace('MALT', 'MALTA')
  df = df.replace('COSTARICA', 'COSTA RICA')
  df = df.replace('COMORO', 'COMOROS')
  df = df.replace('SAUD ARABIA', 'SAUDI ARABIA')
  df = df.replace('PHILIPINES', 'PHILIPPINES')
  df = df.replace('SCOTLAND', 'UNITED KINGDOM')
  df = df.replace('UKRAIN', 'UKRAINE')
  df = df.replace('SOMALI', 'SOMALIA')
  df = df.replace('ECUADO', 'ECUADOR')
  df = df.replace('MONECASQUE', 'MONACO')
  df = df.replace('TRINIDAD TOBACCO', 'TRINIDAD AND TOBAGO')
  df = df.replace('BURGARIA', 'BULGARIA')
  df = df.replace('BOSNIA', 'BOSNIA AND HERZEGOVINA')
  df = df.replace('DJIBOUT', 'DJIBOUTI')
  df['continent'] = df.apply(lambda x: pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(x['country'], cn_name_format="upper")), axis = 1)
  return df

# to rebinning age_group
def binning_age_group(df, bins=3):
  # to 3 bins : -24, 25-44, 45+
  def bin3(age_group):
    if(age_group=='18-24' or age_group=='<18'):
      return '-24'
    if(age_group=='25-44'):
      return '25-44'
    if(age_group=='45-64' or age_group=='65+'):
      return '45+'

  # to 2 bins : -45, 45+
  def bin2(age_group):
    if(age_group=='18-24' or age_group=='<18' or age_group=='25-44'):
      return '-45'
    if(age_group=='45-64' or age_group=='65+'):
      return '45+'

  if(bins == 3):
    df['age_group'] = df.apply(lambda x: bin3(x['age_group']), axis = 1)
  if(bins == 2):
    df['age_group'] = df.apply(lambda x: bin2(x['age_group']), axis = 1)
  return df

# to fill na of some columns
def col_fillna(df):
  # fill travel_with na with 'Alone'
  df['travel_with'].fillna('Alone', inplace=True)
  # fill total_male and total_female na with 1
  df['total_male'].fillna(1, inplace=True)
  df['total_female'].fillna(1, inplace=True)
  # replace 'Widlife Tourism' by 'Wildlife Tourism' in main_activity
  df = df.replace('Widlife Tourism', 'Wildlife Tourism')
  df = df.replace('Yes', 1)
  df = df.replace('No', 0)
  return df

# drop all rows with tour_arrangement=='independent' and any_package=='yes'
def clean_tour_arrangment_package(df):
  # get concerning rows
  df_drop = df.query("tour_arrangement=='Independent' and (package_transport_int=='Yes' or package_transport_int=='Yes' or package_accomodation=='Yes' or package_food=='Yes' or package_transport_tz=='Yes' or package_sightseeing=='Yes' or package_guided_tour=='Yes' or package_insurance=='Yes')")
  # remove rows (its 142 at all concerning)
  df.drop(df_drop.index, inplace=True)
  return df

# add package_acc_food (ie. if the tour package include both accomadation and food )
def add_package_acc_food(df):
  def acc_food(accomodation, food):
    if(accomodation==1 and food==1):
      return 1
    else:
      return 0
  df['package_acc_food'] = df.apply(lambda x: acc_food(x['package_accomodation'], x['package_food']), axis = 1)
  return df

# add package_transport (ie. if the tour package include both international and Tanzania transport )
def add_package_transport(df):
  def transport(international, tz):
    if(international==1 and tz==1):
      return 1
    else:
      return 0
  df['package_transport'] = df.apply(lambda x: transport(x['package_transport_int'], x['package_transport_tz']), axis = 1)
  return df

# add package_sight_guided (ie. if the tour package include both sightseeing and guided_tour )
def add_package_sight_guided(df):
  def sight_guided(sight, guided):
    if(sight==1 and guided==1):
      return 1
    else:
      return 0
  df['package_sight_guided'] = df.apply(lambda x: sight_guided(x['package_sightseeing'], x['package_guided_tour']), axis = 1)
  return df

# add package (ie. if the tour package include all package)
def add_package(df):
  def package(accomodation, food, international, tz, sight, guided):
    if(accomodation==1 and food==1 and international==1 and tz==1 and sight==1 and guided==1):
      return 1
    else:
      return 0
  df['package'] = df.apply(lambda x: package(x['package_accomodation'], x['package_food'],
                                             x['package_transport_int'], x['package_transport_tz'],
                                             x['package_sightseeing'], x['package_guided_tour']), axis = 1)
  return df

# to add total_persons
def add_total_persons(df):
  df['total_persons'] = df['total_male']+df['total_female']
  return df

def encode_cat(X, y, encoders, train=True):
  # Fit encoder and transform the features
  if(train):
    for col in X.columns:
      if(X[col].dtype == 'O'):
        encoders[col].fit(X, y)
  for col in X.columns:
    if(X[col].dtype == 'O'):
      X[col] = encoders[col].transform(X)
  
  return X, encoders

def preprocess_df(df):
  #df = clean_tour_arrangment_package(df)
  df = country_to_continent(df)
  df = binning_age_group(df, bins=3)
  df = col_fillna(df)
  #df = add_package_acc_food(df)
  #df = add_package_transport(df)
  #df = add_package_sight_guided(df)
  df = add_total_persons(df)

  return df

In [11]:
from sklearn.metrics import roc_auc_score as auc
from sklearn.linear_model import LogisticRegression

# to test both MEE and CatBoost for categorical enconding, with simple logistic regression model
# in order to find the best encoding
'''
conclusion : catboost seems to be better than mean estimate encoder
'''
train_cp = train.copy()
train_proc = preprocess_df(train_cp)
y = train_proc['cost_category']
y = y.map(lambda x: 0 if x=='Lower Cost' else 1 if x=='Low Cost' else 2 if x=='Normal Cost' else 3 if x=='High Cost' else 4 if x=='Higher Cost' else 5)
X = train_proc.copy()
drop_cols = ['Tour_ID', 'country', 'cost_category']
X.drop(drop_cols, axis=1, inplace=True)

xtrain, xvalid, ytrain, yvalid = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 42,
    shuffle = True,
    stratify = y.values    
)

mee_encoder = ce.MEstimateEncoder(m=10.0)
cbe_encoder = ce.cat_boost.CatBoostEncoder()
encoders = [mee_encoder, cbe_encoder]

for encoder in encoders:
  print(encoder)
  xtrain_enc, enc = encode_cat(xtrain, ytrain, encoder, train=True)
  xvalid_enc, enc = encode_cat(xvalid, yvalid, enc, train=False)
  
  lr = LogisticRegression(max_iter=1000)
  lr.fit(xtrain_enc, ytrain)
  train_preds = lr.predict(xtrain_enc)
  valid_preds = lr.predict(xvalid_enc)
  print('MSE: train, valid', mean_squared_error(ytrain, train_preds, squared=False), mean_squared_error(yvalid, valid_preds, squared=False))

In [ ]:
# Grid search : find optimal parameters

import category_encoders as ce

# load files
train = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/Test.csv')
variableDefinitions = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/VariableDefinitions.csv')
sampleSubmission = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/SampleSubmission.csv')
train = train.fillna(-1)
test = test.fillna(-1)
train = country_to_continent(train)
train = add_total_persons(train)
train = add_package_acc_food(train)
train = add_package_transport(train)
train = add_package_sight_guided(train)
test = country_to_continent(test)
test = add_total_persons(test)
test = add_package_acc_food(test)
test = add_package_transport(test)
test = add_package_sight_guided(test)

def modelfit(alg, dtrain, target,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain.values, label=target.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='mlogloss', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
        print(cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain, target,eval_metric='logloss')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    dtrain_predprob = alg.predict(dtrain)
    print('MSE', mean_squared_error(target, dtrain_predictions.argmax(axis=1), squared=False))
    print('log_loss', log_loss(target, dtrain_predictions))

# scorer for gridsearch score
def log_loss_scorer(clf, X, y):
  y_pred = clf.predict(X)
  return -1 * log_loss(y, y_pred) # on multiplie par -1 pour que la fonction renvoie la plus petite erreur lorsqu'elle est applelee dans le grid_search (grid search calcule les params avec le score le plus eleve)

# train ds
train_enc = train.copy()
target_col = 'cost_category'
train_enc[target_col] = train_enc[target_col].map(lambda x: 0 if x=='Lower Cost' else 1 if x=='Low Cost' else 2 if x=='Normal Cost' else 3 if x=='High Cost' else 4 if x=='Higher Cost' else 5)
target = train_enc[target_col]
train_enc.drop(['Tour_ID'], axis=1, inplace=True)
train_enc.drop('cost_category', axis=1, inplace=True)

train_enc_final = train_enc.copy()

# test ds
test_enc = test.drop(columns=['Tour_ID'])
  
dict_enc = {col:ce.MEstimateEncoder(m=10.0) for col in train_enc_final.columns} 

for col in train_enc_final.columns:
  if train_enc_final[col].dtype == 'O' :
    train_enc_final[col] = dict_enc[col].fit_transform(train_enc_final[col], target)

# step 1 : find the best n_estimators value --- found = 119
'''
model = XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.8,
                     max_depth=5, min_child_weight = 1, gamma = 0., colsample_bytree = 0.8,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0., learning_rate=0.11, n_estimators=1000)
modelfit(model, train_enc_final, target)
'''

# step 2.a: Tune max_depth and min_child_weight
# result: {'max_depth': 5, 'min_child_weight': 5} - time: 13 min 5s
'''
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.8,
                     max_depth=5, min_child_weight = 1, gamma = 0., colsample_bytree = 0.8,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0., learning_rate=0.11, n_estimators=119), 
 param_grid = param_test1, scoring=log_loss_scorer,n_jobs=4, cv=5, error_score='raise')
gsearch1.fit(train_enc_final, target)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_
'''

# step 2.b:  we have get max_depth=5, min_child=5 from previous research. let us try with range [4,5,6] to get best 
# 'max_depth': 5, 'min_child_weight': 6
'''
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch1 = GridSearchCV(XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.8,
                     max_depth=5, min_child_weight = 2, gamma = 0., colsample_bytree = 0.8,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0., learning_rate=0.11, n_estimators=119), 
 param_grid = param_test2, scoring=log_loss_scorer,n_jobs=4, cv=5, error_score='raise')
gsearch1.fit(train_enc_final, target)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_
'''

# step 2.c: we get 6 as best min_child_weight. we have not tested vlaues greater than 6. 
#min_child_weight : 10
'''
param_test2c = {
 #'min_child_weight':[6,8,10,12]
 'min_child_weight':[9,10,11]
}
gsearch1 = GridSearchCV(XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.8,
                     max_depth=5, min_child_weight = 2, gamma = 0., colsample_bytree = 0.8,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0., learning_rate=0.11, n_estimators=119), 
 param_grid = param_test2c, scoring=log_loss_scorer,n_jobs=4, cv=5, error_score='raise')
gsearch1.fit(train_enc_final, target)
modelfit(gsearch1.best_estimator_, train_enc_final, target)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_
'''

# step3: tune gamma
# 'gamma': 0.1
'''
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.8,
                     max_depth=5, min_child_weight = 10, gamma = 0., colsample_bytree = 0.8,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0., learning_rate=0.11, n_estimators=119), 
 param_grid = param_test3, scoring=log_loss_scorer,n_jobs=4, cv=5, error_score='raise')
gsearch3.fit(train_enc_final, target)
modelfit(gsearch3.best_estimator_, train_enc_final, target)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_
'''
# recalibrate n_estimators
#n_estimators = 144
'''
model = XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.8,
                     max_depth=5, min_child_weight = 10, gamma = 0.1, colsample_bytree = 0.8,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0., learning_rate=0.11, n_estimators=1000)
modelfit(model, train_enc_final, target)
'''

# step 4 : tune col_sample_bytree and subsample
# 'colsample_bytree': 0.6, 'subsample': 0.6
'''
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.8,
                     max_depth=5, min_child_weight = 10, gamma = 0.1, colsample_bytree = 0.8,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0., learning_rate=0.11, n_estimators=144), 
 param_grid = param_test4, scoring=log_loss_scorer,n_jobs=4, cv=5, error_score='raise')
gsearch4.fit(train_enc_final, target)
modelfit(gsearch4.best_estimator_, train_enc_final, target)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_
'''
# 'colsample_bytree': 0.6, 'subsample': 0.65
'''
param_test4 = {
 'subsample':[i/100.0 for i in range(55,70,5)],
 'colsample_bytree':[i/100.0 for i in range(55,70,5)]
}
gsearch4 = GridSearchCV(XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.6,
                     max_depth=5, min_child_weight = 10, gamma = 0.1, colsample_bytree = 0.6,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0., learning_rate=0.11, n_estimators=144), 
 param_grid = param_test4, scoring=log_loss_scorer,n_jobs=4, cv=5, error_score='raise')
gsearch4.fit(train_enc_final, target)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_
'''

'''
param_test5 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch5 = GridSearchCV(XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.65,
                     max_depth=5, min_child_weight = 10, gamma = 0.1, colsample_bytree = 0.6,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0.01, learning_rate=0.11, n_estimators=144), 
 param_grid = param_test5, scoring=log_loss_scorer,n_jobs=4, cv=5, error_score='raise')
gsearch5.fit(train_enc_final, target)
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_
'''

# recalibrate n_estimators
#n_estimators = 115, log_loss=0.9638
model = XGBRegressor(random_state=42, seed=27, objective='multi:softprob', num_class=6, subsample=0.65,
                     max_depth=5, min_child_weight = 10, gamma = 0.1, colsample_bytree = 0.6,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0.01, learning_rate=0.11, n_estimators=1000)
modelfit(model, train_enc_final, target)

115
MSE 0.938794856712211
log_loss 0.9638221557029167


In [10]:
# solution
'''
- target encoding : normal 2 - higher 4 - high 3 - lower 0 - low 1 - highest 5
- categorical encoding: MestimateEncoder
- predictors : xgboost with optimal params (grid search have been done before) 
'''
import category_encoders as ce

# load files
train = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/Test.csv')
variableDefinitions = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/VariableDefinitions.csv')
sampleSubmission = pd.read_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/SampleSubmission.csv')
train = train.fillna(-1)
test = test.fillna(-1)
train = country_to_continent(train)
train = add_total_persons(train)
train = add_package_acc_food(train)
train = add_package_transport(train)
train = add_package_sight_guided(train)
test = country_to_continent(test)
test = add_total_persons(test)
test = add_package_acc_food(test)
test = add_package_transport(test)
test = add_package_sight_guided(test)


#train = binning_age_group(train, bins=3)
#train = col_fillna(train)
#test = col_fillna(test)

# train ds
train_enc = train

encoding_dict = dict()
col = 'cost_category'
value = train_enc[col].value_counts().index.tolist()
dict_value = {value:count for value,count in zip(value, range(1, len(value)+1))}
encoding_dict[col] = dict_value
#train_enc[col] = train_enc[col].map(lambda x: dict_value.get(x)) - 1
train_enc[col] = train_enc[col].map(lambda x: 0 if x=='Lower Cost' else 1 if x=='Low Cost' else 2 if x=='Normal Cost' else 3 if x=='High Cost' else 4 if x=='Higher Cost' else 5)

dict_enc = {col:ce.MEstimateEncoder(m=10.0) for col in train_enc.columns}
#dict_enc = {col:ce.cat_boost.CatBoostEncoder() for col in train_enc.columns}

for col in train_enc.columns:
  if train_enc[col].dtype == 'O' :
    train_enc[col] = dict_enc[col].fit_transform(train_enc[col], train_enc['cost_category'])
train_enc_final = train_enc.copy()
train_enc_final.drop(['Tour_ID'], axis=1, inplace=True)
train_enc_final.drop('cost_category', axis=1, inplace=True)

# test ds
test_enc = test.drop(columns=['Tour_ID'])
for col in test_enc.columns:
  if test_enc[col].dtype == 'O':
    test_enc[col] = dict_enc[col].transform(test_enc[col])
test_enc_final = test_enc

xtrain, xvalid, ytrain, yvalid = train_test_split(
    train_enc_final,
    train_enc['cost_category'],
    test_size = 0.15,
    random_state = 42,
    shuffle = True,
    stratify = train['cost_category'].values    
)

xtest = test_enc_final

model = XGBRegressor(seed=27, objective='multi:softprob', eta=0.005, num_class=6, subsample=0.65,
                     max_depth=5, min_child_weight = 10, gamma = 0.1, colsample_bytree = 0.6,  nthread=4, 
                     scale_pos_weight = 1, reg_alpha=0.01, learning_rate=0.11, n_estimators=115)

model.fit(xtrain, ytrain)
train_preds = model.predict(xtrain)
valid_preds = model.predict(xvalid)
test_preds = model.predict(xtest)
print("train")
print('MSE', mean_squared_error(ytrain, train_preds.argmax(axis=1), squared=False))
print('log_loss', log_loss(ytrain, train_preds))
print("valid")
print('MSE', mean_squared_error(yvalid, valid_preds.argmax(axis=1), squared=False))
print('log_loss', log_loss(yvalid, valid_preds))

preds0 = test_preds[:, 0]
preds1 = test_preds[:, 1]
preds2 = test_preds[:, 2]
preds3 = test_preds[:, 3]
preds4 = test_preds[:, 4]
preds5 = test_preds[:, 5]

sampleSubmission['Normal Cost'] = preds2
sampleSubmission['Higher Cost'] = preds4
sampleSubmission['High Cost'] = preds3
sampleSubmission['Lower Cost'] = preds0
sampleSubmission['Low Cost'] = preds1
sampleSubmission['Highest Cost'] = preds5
sampleSubmission.to_csv('/content/drive/MyDrive/Zindi/ai4d-lab-tanzania-tourism-classification-challenge/sub_final3.csv', index = False)

train
MSE 0.9377645844059436
log_loss 0.9590592325025031
valid
MSE 0.9503146756253603
log_loss 1.0440353382773353
